<html>
<body>
<h1>Explainable_Medical_Image_Classifier_based_on_Human_Diagnosis_Behaviour_PT</h1>
<h4>Author: Anshu Garg </h4>
<h4>Matriculation Number: 414686</h4>
</body>
</html>

In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

E:\Anaconda\envs\acvdl\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'E:\Anaconda\envs\acvdl\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.40, 0.45, 0.45], std=[0.21, 0.23, 0.22]),
])

In [3]:
device=torch.device('cuda')

In [4]:
#Downloading training data
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)

#Downloading test data
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

#Class labels
classes = ('plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#Now using the AlexNet
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

#Model description
AlexNet_model.eval()

Using cache found in C:\Users\Anshu Garg/.cache\torch\hub\pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
#Updating the second classifier
AlexNet_model.classifier[4] = torch.nn.Linear(4096,1024)

#Updating the third and the last classifier that is the output layer of the network. Make sure to have 10 output nodes if we are going to get 10 class labels through our model.
AlexNet_model.classifier[6] = torch.nn.Linear(1024,10)

In [7]:
AlexNet_model.eval()
AlexNet_model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
criterion = torch.nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = torch.optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = AlexNet_model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training of AlexNet')

[1,  2000] loss: 1.200
[1,  4000] loss: 0.873


In [ ]:
torch.save(AlexNet_model.state_dict(),'./data1')

In [ ]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = AlexNet_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
AlexNet_model.load_state_dict(torch.load('./data1'))
# Splitting the alexnet model
#1st model
module_1=torch.nn.Sequential(*list(AlexNet_model.features.children())[:-3])
module_2=torch.nn.Sequential(*list(AlexNet_model.features.children())[:-1])

In [ ]:
module_1.to(device)
module_2.to(device)

In [ ]:
from sklearn.svm import SVC
from sklearn import svm
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
print("train_data.data",train_data.data.shape)
train_data_numpy=torch.from_numpy(train_data.data)
train_DT = train_data_numpy.permute(0, 3, 1, 2)
print("train_data_numpy",train_data_numpy.shape)

train_final=train_DT.float().to(device)
print(train_DT.shape)

test_input=torch.from_numpy(test_data.data)
test_DT = test_input.permute(0, 3, 1, 2)
test_final=test_DT.float().to(device)
print("test_DT",test_DT.shape)

In [ ]:
output_1=module_1(train_final)   #50000, 256, 1, 1
output_2=module_2(train_final)   #50000, 256, 1, 1

In [ ]:
output_1=output_1.cpu().detach().numpy()
output_2=output_2.cpu().detach().numpy()

In [ ]:
final_out_1=output_1.reshape(output_1.shape[0], output_1.shape[1]*output_1.shape[2]*output_1.shape[3])
final_out_2=output_2.reshape(output_2.shape[0], output_2.shape[1]*output_2.shape[2]*output_2.shape[3])
print(final_out_1.shape)
print(final_out_2.shape)

labels_array=np.array(train_data.targets)
print(labels_array.flatten().shape)

In [ ]:
#train SVM 1
svm1 = svm.LinearSVC()
svm1.fit(final_out_1, labels_array)

In [ ]:
#train SVM 2
svm2 = svm.LinearSVC()
svm2.fit(final_out_2, labels_array)

In [ ]:
#Find intermediate output of Alexnet architecture
test_output_1=module_1(test_final)
test_output_2=module_2(test_final)

In [ ]:
test_output_1=test_output_1.cpu().detach().numpy()
test_output_2=test_output_2.cpu().detach().numpy()

test_final_out_1=test_output_1.reshape(test_output_1.shape[0], test_output_1.shape[1]*test_output_1.shape[2]*test_output_1.shape[3])
test_final_out_2=test_output_2.reshape(test_output_2.shape[0], test_output_2.shape[1]*test_output_2.shape[2]*test_output_2.shape[3])

In [ ]:
#predicting first SVM
svm_test_pred = svm1.predict(test_final_out_1) 

#predicting second SVM
svm_test_pred_2 = svm2.predict(test_final_out_2) 

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
test_out = np.array(test_data.targets)
accuracy = svm1.score(test_final_out_1, test_out)
print("Accuracy of SVM1 - ", accuracy)
print("Classification Report for SVM1")
cls_rpt = classification_report(test_out, svm_test_pred) 
print(cls_rpt)

In [ ]:
accuracy2 = svm2.score(test_final_out_2, test_out)
print("Accuracy of SVM2 - ", accuracy2)
print("Classification Report for SVM2")
cls_rpt2 = classification_report(test_out, svm_test_pred_2) 
print(cls_rpt2)